In [30]:
print(__doc__)

import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np

from sklearn import datasets, svm
from sklearn.cross_validation import StratifiedKFold
from sklearn.externals.six.moves import xrange
from sklearn.mixture import GaussianMixture
from sklearn.metrics import precision_score, recall_score, accuracy_score, roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, VotingClassifier

from tqdm import tqdm_notebook as tqdm
from scipy import interp
import pickle
import librosa
import re
import os

Automatically created module for IPython interactive environment


In [31]:
# settings
dataPath = ".\\wav"
classes = 40

In [32]:
# find personId from the file path
def getPerson(filePath):
    matchObj = re.match(r'.*id(\d+).*', filePath)
    return int(matchObj.group(1))-10270

# sample class
class VoiceSample:
    path = ''
    personId = -1
    feature = np.empty([13, 0])
    
    def __init__(self, path, id = -1):
        self.path = path
        
        if id == -1:
            self.personId = getPerson(path)
        else:
            self.personId = id
        
        self.getAvgMFCC()
        
    def __str__(self):
        return self.path
    
    def getAvgMFCC(self):
        y, sr = librosa.load(self.path,sr=None)
        mfccs = librosa.feature.mfcc(y=y, sr=sr,n_mfcc=13)
        # self.feature = np.append(self.feature, mfccs, axis = 1)
        self.feature = np.average(mfccs, 1).reshape([1, 13])

def gci(filepath, res):
    files = os.listdir(filepath)
    for fi in tqdm(files):
        fi_d = os.path.join(filepath,fi)
        if os.path.isdir(fi_d):
            gci(fi_d, res)                  
        else:
            voice = VoiceSample(fi_d)
            res[0] = np.append(res[0], voice.feature, axis = 0) 
            res[1].append(voice.personId)
    
def getList(filepath):
    res = [np.empty([0, 13]),[]]
    gci(filepath, res)
    return res

#load background data
data = getList(dataPath)
# print data[0].shape, len(data[1])
# voice = VoiceSample(r'C:\Users\aimin\SpeakerRecognition\wav\id10270\5r0dWxy17C8\00001.wav')

In [39]:
pickle.dump(data, open("newData", "wb"))

In [76]:
data = pickle.load(open("newData", "rb"))

In [77]:
print data[0].shape, len(data[1])

(4874L, 13L) 4874


In [91]:
# new data
fileList = [r'.\\input1.wav', r'.\\input2.wav']
for fi_d in fileList:  
    tarClass = 40
    y, sr = librosa.load(fi_d,sr=None)
    mfccs = librosa.feature.mfcc(y=y, sr=sr,n_mfcc=13).T
    
    for i in range(mfccs.shape[0]/50):
        toAdd = np.average(mfccs[i*50 : i*50 + 50], 0).reshape([1,13])
        data[0] = np.append(data[0], toAdd, 0)
        data[1].append(tarClass)
    #data[0] = np.append(data[0], mfccs.T, axis = 0)
#     for i in range(mfccs.shape[1]):
#         data[1].append(tarClass)
print data[0].shape, len(data[1])

(5227L, 13L) 5227


In [92]:
clf =svm.SVC(kernel = "rbf", gamma=0.001)
clf.fit(data[0].tolist(), data[1])

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [100]:
testFile = r'.\juesha.wav'
y, sr = librosa.load(testFile,sr=None)
mfccs = librosa.feature.mfcc(y=y, sr=sr,n_mfcc=13).T
mean = np.average(mfccs, 0)
print clf.predict(mean.reshape([1,13]))

[40]
